# Infinite slope analysis

## Initialise code

First, we need to load some packages and functions.

In [ ]:
# load packages and functions
import numpy as np
import matplotlib.pyplot as plt
from pyrootmemo.materials import Soil
from pyrootmemo.geometry import SoilProfile, FailureSurface
from pyrootmemo.models import Wwm, Fbm, Rbmw, Waldron
# define parameter object (value + unit), used later to define soil, root and geometry parameters
from pyrootmemo.helpers import units
from collections import namedtuple
Parameter = namedtuple("parameter", "value unit")

## Define soil profile and failure surface

First, define a series of soils

In [ ]:
# 'typical' clayey material
clay = Soil(
    'Clay',
    cohesion = Parameter(10, 'kPa'),
    friction_angle = Parameter(17.5, 'degrees'),
    unit_weight_bulk = Parameter(16, 'kN/m^3'),
    unit_weight_saturated = Parameter(18, 'kN/m^3')
    )

# 'typical' sandy material
sand = Soil(
    'Sand',
    cohesion = Parameter(0, 'kPa'),
    friction_angle = Parameter(35, 'degrees'),
    unit_weight_bulk = Parameter(18, 'kN/m^3'),
    unit_weight_saturated = Parameter(20, 'kN/m^3')
    )

Combine soils, and information about the water table, into a soil profile

In [ ]:
soil_profile = SoilProfile(
    [clay, sand],                           # clay on top of sand
    depth = Parameter([2.0, 10.0], 'm'),    # 2 m thick clay layer, 8 m thick sand layer
    groundwater_table = Parameter(1.5, 'm') # groundwater table at 1.5 m below ground surface
    )

Define the geometry of the slope and failure surface

In [ ]:
failure_surface = FailureSurface(
    depth = Parameter(3, 'm'),           # depth of failure surface below ground surface
    orientation = Parameter(20, 'deg')   # slope angle of the failure surface 
    )    

Set an amount of root cohesion at the failure surface

In [ ]:
root_cohesion = 2.0 * units('kPa')

## Infinite slope calculation

Calculate soil and water stresses at the failure surface

In [ ]:
# pore pressure at failure surface
pore_pressure = soil_profile.calc_pore_pressure(
    failure_surface.depth,
    direction = failure_surface.orientation
    )

# total vertical soil stress, at failure surface
total_vertical_stress = soil_profile.calc_vertical_stress(failure_surface.depth)

# print results
print('pore pressure:', pore_pressure.to('kPa'))
print('total vertical stress:', total_vertical_stress.to('kPa'))

Get the soil properties (cohesion, friction angle) at the failure surface

In [ ]:
# soil friction angle and cohesion at failure surface
soil_cohesion = soil_profile.get_soil(failure_surface.depth).cohesion
friction_angle = soil_profile.get_soil(failure_surface.depth).friction_angle

Calculate the stabilising and destabilising forces

In [ ]:
# driving stresses
stress = (
    total_vertical_stress
    * np.sin(failure_surface.orientation)
    * np.cos(failure_surface.orientation)
    )

# resisting strength
strength = (
    soil_cohesion
    + root_cohesion
    + np.tan(friction_angle) * (
        total_vertical_stress * np.cos(failure_surface.orientation)**2 
        - pore_pressure
        )
    )

# print results
print('Destabilising stresses:', stress.to('kPa'))
print('Resisting strengths:', strength.to('kPa'))

Calculate the factor of safety

In [ ]:
FoS = strength / stress
print('Factor of safety:', FoS.to(''))

## Explore!

How is the factor of safety affected by:
* Root 'cohesion'
* Soil properties and profile
* Groundwater conditions